# Infraestructure Manager revenue maximization with GSA

## 0. Load libraries

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from src.entities import GSA

from copy import deepcopy
from math import cos, pi, e
from typing import List, Mapping, Union

## 1. Define corridor

As an example, we will use the Spanish south high-speed railway corridor.



In [2]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                        },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

### 1.1. Get lines from corridor

In [3]:
# Get lines from corridor

def get_lines(corridor: dict, path: Union[list, None]=None) -> list:
    """
    Get all the lines in the corridor
    
    Args:
        corridor (dict): dictionary with the corridor structure
        path (list, optional): list of nodes
    
    Returns:
        list of lines
    """
    if path is None:
        path = []

    lines = []
    for node, child in corridor.items():
        new_path = path + [node]
        if not child:  # If the node has no children, it is a leaf
            lines.append(new_path)
        else:
            lines.extend(get_lines(child, new_path))  # If the node has children, we call the function recursively

    return lines

get_lines(corridor)

[['MAD', 'CIU', 'COR', 'SEV', 'CAD'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']]

### 1.2. Sample a random line, and a random route from the line

In [4]:
def sample_line(lines: list) -> list:
    """
    Sample a random line from the list of lines
    
    Args:
        lines (list): list of lines
    
    Returns:
        list: random line
    """
    return lines[np.random.randint(len(lines))] 

def sample_route(line: list) -> list:
    """
    Sample a random route from line
    
    Args:
        line (list): list of stations
        
    Returns:
        list: random route
    """
    return line[np.random.randint(0, len(line)-1):]

lines = get_lines(corridor)

line = sample_line(lines)
print(f"Sampled line: {line}")

# Sample a random route from line (at least two stations)
route = sample_route(line)
print(f"Sampled route: {route}")

Sampled line: ['MAD', 'CIU', 'COR', 'SEV', 'CAD']
Sampled route: ['MAD', 'CIU', 'COR', 'SEV', 'CAD']


### 1.3 Generate random timetable for a route

- Times in minutes.
- Initial time is randomized between 0 and 24*60 (24 hours).
- The time between stations is also randomized between 30 and 120 minutes.
- The time to dwell at each station is randomized between 2 and 8 minutes.

In [5]:
def get_timetable(route: list) -> dict:
    """
    Generate random timetable for route r
    
    Args:
        route (list): list of stations
    
    Returns:
        dict: timetable
    """
    timetable = {}
    AT = np.random.randint(0, 24*60)
    DT = AT
    for i, sta in enumerate(route):
        if i == 0 or i == len(route)-1:
            timetable[sta] = (AT, AT)
        else:
            timetable[sta] = (AT, DT)
            
        AT += np.random.randint(30, 120)
        DT = AT + np.random.randint(2, 8)
        
    return timetable

get_timetable(route)

{'MAD': (1331, 1331),
 'CIU': (1390, 1395),
 'COR': (1449, 1455),
 'SEV': (1510, 1516),
 'CAD': (1610, 1610)}

## 2. Generate as many requested services as needed

In [6]:
# Generate random requested timetable in corridor for a day t

def get_schedule_request(n_services: int) -> dict:
    """
    Generate random timetable
    
    Args:
        n_services (int): number of services
    
    Returns:
        dict: timetable
    """
    return {i: get_timetable(sample_route(sample_line(lines))) for i in range(1, n_services+1)}

# Generate random schedule
schedule = get_schedule_request(3)
schedule

{1: {'PGE': (337, 337), 'ANT': (371, 373), 'GRA': (437, 437)},
 2: {'ANT': (741, 741), 'GRA': (804, 804)},
 3: {'ANT': (36, 36), 'GRA': (127, 127)}}

## 3. Define feasibility function

A service $ i $ is feasible (can be scheduled and, therefore, $ S_i $ take the value 1) if the following condition is met:

$ | DT_{ij} - DT_{kj} | \geq security\_gap $

Where:
- $ DT_{ij} $ is the departure time for service $ i $ from station $ j $.
- $  DT_{kj} $ is the departure time for service $ k $ from the same station $ j $.

Inequality must be met for all services ($ k \neq j $) that also operate at station $ j $ for service $ i $ to be feasible.

1) Requests from the Railway Undertakings (RUs) are considered as a set of services that can be scheduled or not. The decision variable $ S_i $ is binary and takes the value 1 if the service $ i $ is scheduled and 0 otherwise.

Service ID |  RU   |          Trip          |              Requested schedule              | 
:---: |:-----:|:----------------------:|:--------------------------------------------:|
1 | Renfe |   `[MAD, BAR, FIG]`    |      `[(0, 0), (148, 152), (180, 180)]`      |
2 | Ouigo | `[MAD, ZAR, BAR, FIG]` | `[(8, 8), (28, 30), (165, 167), (210, 210)]` |
3 | Iryo  |   `[MAD, BAR, FIG]`    |     `[(30, 30), (180, 182), (225, 225)]`     |


2) Get conflict matrices for each service. A conflict matrix is a binary matrix of 0's and 1's. A 1 appears in the outputs of services (different from the one we are analyzing) that interferes with a station of the service in question.

Service 1: Has an interference with the service 2 at the departure from Madrid.

Service ID | MAD | BAR | FIG |
:---: | :---: | :---: | :---: |
1 | 0 | 0 | 0 |
2 | 1 | 0 | 0|
3 | 0 | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_1} = (S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) = 0 $ \\

Service 2: Has an interference with the service 1 at the departure from Madrid.

ID servicio | MAD | ZAR | BAR | FIG |
:---: | :---: | :---: | :---: | :---: |
1 | 0 | 0 | 0 | 0 |
2 | 1 | 0 | 0 | 0|
3 | 0 | 0 | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_2} = (S_1 \cdot 1 + S_1 \cdot 0 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) = 0 $ \\

Service 3: Does not have any interference with the other services.

ID servicio | MAD | BAR | FIG |
:---: |:---:| :---: | :---: |
1 |  0  | 0 | 0 |
2 |  0  | 0 | 0|
3 |  0  | 0 | 0 |

In order to be feasible, the following condition must be met:

$ R_{S_2} = (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) + (S_1 \cdot 0 + S_1 \cdot 0 + S_3 \cdot 0) +(S_1 \cdot 0 + S_1 \cdot 1 + S_3 \cdot 0) \leq 0 $ \\

We can define a general restriction to check if the timetable is feasible or not:

$ S_1 \cdot R_{S_1} + S_2 \cdot R_{S_2} + S_2 \cdot R_{S_2} \leq 0$

This equation can be read as follows:

- If a service $ S_i $ is planned, then its restrictions $ R_{S_i} $ must be met. If the service $ S_i $ is not planned, its own interferences do not have to be taken into account.
- For the set of schedules to be feasible, there must be no conflict between any of the planned services (all the $ S_i $ that take the value $ 1 $)

In [48]:
from functools import lru_cache

class RevenueMaximization:
    """
    Class for the revenue maximization problem.
    """
    def __init__(self,
                 requested_schedule: Mapping[int, Mapping[str, Union[int, int]]],
                 revenue: Mapping[int, Mapping[str, Union[float, int]]],
                 safe_headway: int=10
                 ) -> None:
        """
        Constructor
        
        Args:
            requested_schedule (dict): requested schedule
        """
        self.requested_schedule = requested_schedule 
        self.revenue = revenue
        self.updated_schedule = deepcopy(requested_schedule)
        self.requested_times = self.get_real_vars()
        self.real_boundaries = self.get_real_boundaries()
        self.boundaries = {'real': self.real_boundaries, 'discrete': np.array([])}
        self.safe_headway = safe_headway
        self.feasible_schedules = []
        self.scheduled_trains = np.zeros(len(self.requested_schedule))
    
    @staticmethod
    def penalty_function(x: float, k: int) -> float:
        """
        Penalty function
        
        Args:
            x (float): x
            k (int): k
        
        Returns:
            float: penalty
        """
        return 1 - e**(-k * x**2) * ( (1/2) * cos(pi * x) + (1/2) )
    
    def get_revenue(self, 
                    solution: Mapping[str, List[Union[float, int]]],
                    update_schedule: bool=True) -> int:
        """
        Get IM revenue.
        
        Args:
            solution (dict): solution
        
        Returns:
            float: IM revenue
        """
        if update_schedule:
            self.update_schedule(solution)
        S_i = solution['discrete']
        
        im_revenue = 0
        for i, service in enumerate(self.requested_schedule):
            k = self.revenue[service]['k']
            departure_station = list(self.requested_schedule[service].keys())[0]
            departure_time_delta = abs(self.updated_schedule[service][departure_station][1] - self.requested_schedule[service][departure_station][1])
            tt_penalties = []
            for j, stop in enumerate(self.requested_schedule[service].keys()):
                if i == 0 or i == len(self.requested_schedule[service]) - 1:
                    continue
                tt_penalty = self.penalty_function(abs(self.updated_schedule[service][stop][1] - self.requested_schedule[service][stop][1]) / self.safe_headway, k)
                tt_penalties.append(tt_penalty * self.revenue[service]['tt_max_penalty'])
            dt_penalty = self.penalty_function(departure_time_delta / self.safe_headway, k) * self.revenue[service]['dt_max_penalty']
            im_revenue += self.revenue[service]['canon'] * S_i[i] - dt_penalty *  S_i[i] - np.sum(tt_penalties) *  S_i[i]
            
        return im_revenue
    
    def update_schedule(self, solution: Mapping[str, np.ndarray]):
        """
        Update schedule with the solution
        
        Args:
            solution (dict): solution
        """
        times = solution["real"] if solution["real"].size else self.get_real_vars()
        s_idx = 0
        for i, service in enumerate(self.updated_schedule):
                for j, stop in enumerate(self.updated_schedule[service]):
                    if j == 0 or j == len(self.updated_schedule[service]) - 1:
                        self.updated_schedule[service][stop] = (times[s_idx], times[s_idx])
                        s_idx += 1
                    else:
                        self.updated_schedule[service][stop] = (times[s_idx], times[s_idx+1])
                        s_idx += 2
    
    def _departure_time_feasibility(self, scheduling) -> bool:
        """
        Check if there are any conflicts with the departure times.
        
        Args:
            solution (dict): solution
        
        Returns:
            bool: True if the departure time is feasible, False otherwise
        """
        S_i = scheduling
        security_array = []

        # Get conflicts between services
        for service in self.updated_schedule:
            service_sec_arr = []  # Build security matrix for service (columns are stops, rows are other services)
            for service_k in self.updated_schedule:
              service_sec_row = [] 
              for stop in self.updated_schedule[service]:
                if service_k == service or stop not in self.updated_schedule[service_k]:
                  service_sec_row.append(0)
                  continue

                if abs(self.updated_schedule[service][stop][1] - self.updated_schedule[service_k][stop][1]) < self.safe_headway:
                  service_sec_row.append(1)
                else:
                  service_sec_row.append(0)
              service_sec_arr.append(service_sec_row)

            security_array.append(np.array(service_sec_arr))
            
        # Get array of conflicts. If there is a conflict, the dot product will be different from zero
        if not S_i.dot(np.array([np.sum(S_i.dot(ssa)) for ssa in security_array])):
            return True
        return False
    
    def _stop_times_feasibility(self, scheduling):
        """
        Check if the travel times are feasible. In order to be feasible, the travel times must be greater than the requested ones.
        
        Args:
            solution (dict): solution
        
        Returns:
            bool: True if the travel times are feasible, False otherwise
        """
        S_i = scheduling
        for i, service in enumerate(self.requested_schedule):
            if S_i[i] == 0:
                continue
            original_service_times = tuple(self.requested_schedule[service].values())
            updated_service_times = tuple(self.updated_schedule[service].values())
            for j in range(1, len(original_service_times)-1):
                original_st = original_service_times[j][1] - original_service_times[j][0]
                updated_st = updated_service_times[j][1] - updated_service_times[j][0]
                if updated_st < original_st:
                    return False
        return True
        
    def _travel_times_feasibility(self, scheduling) -> bool:
        """
        Check if the travel times are feasible. In order to be feasible, the travel times must be greater than the requested ones.
        
        Returns:
            bool: True if the travel times are feasible, False otherwise
        """
        S_i = scheduling
        for i, service in enumerate(self.requested_schedule):
            if S_i[i] == 0:
                continue
            original_service_times = tuple(self.requested_schedule[service].values())
            updated_service_times = tuple(self.updated_schedule[service].values())
            for j in range(len(original_service_times)-1):
                original_tt = original_service_times[j+1][0] - original_service_times[j][1]
                updated_tt = updated_service_times[j+1][0] - updated_service_times[j][1]
                if updated_tt < original_tt:
                    return False
        return True
    
    def _feasible_boundaries(self, solution) -> bool:
        """
        Check if the solution is within the boundaries
        
        Args:
            solution (dict): solution
        
        Returns:
            bool: True if the solution is within the boundaries, False otherwise
        """
        for i, rv in enumerate(solution["real"]):
            if rv < self.real_boundaries[i][0] or rv > self.real_boundaries[i][1]:
                return False 
        return True
         
    def is_feasible(self,
                    timetable: Mapping[str, np.array],
                    scheduling: np.array,
                    update_schedule: bool=True
                    ) -> bool:
        """
        Check if the solution is feasible
        
        Args:
            timetable (Mapping[str, np.array]): solution obtained from the optimization algorithm
            scheduling (np.array): scheduling
        
        Returns:
            bool: True if the solution is feasible, False otherwise
        """
        if update_schedule:
            self.update_schedule(timetable)
            
        if not self._feasible_boundaries(timetable):
            return False
        
        if (self._departure_time_feasibility(scheduling) and 
                self._travel_times_feasibility(scheduling) and 
                self._stop_times_feasibility(scheduling)):
            return True
        return False
    
    def get_real_vars(self):
        """
        Get real variables
        
        Returns:
            list: real variables with requested schedule times.
        """
        real_vars = []
        for service in self.requested_schedule:
            for i, stop in enumerate(self.requested_schedule[service]):
                if i == 0 or i == len(self.requested_schedule[service]) - 1:
                    real_vars.append(self.requested_schedule[service][stop][0])
                else: 
                    real_vars.append(self.requested_schedule[service][stop][0])
                    real_vars.append(self.requested_schedule[service][stop][1])

        return real_vars
    
    def get_real_boundaries(self):
        """
        Get real boundaries
        
        Returns:
            list: real boundaries
        """
        real_vars = self.get_real_vars()
        real_boundaries = []
        for rv in real_vars:
            real_boundaries.append((rv-10, rv+10))

        return np.array(real_boundaries)
    
    def update_feasible_schedules(self, timetable: Mapping[str, np.array]):
        """
        Get feasible scheduling
        
        Args:
            timetable (Mapping[str, np.array]): timetable
        """
        def truth_table(dim: int):
            if dim < 1:
                return [[]]
            sub_tt = truth_table(dim-1)
            return [row + [val] for row in sub_tt for val in [0, 1]]
        
        self.update_schedule(timetable)
        
        train_combinations = truth_table(len(self.requested_schedule))
        self.feasible_schedules = list(filter(lambda schdl: self.is_feasible(timetable, np.array(schdl), update_schedule=False), train_combinations))
    
    def get_best_schedule(self, timetable):
        """
        Get best schedule
        
        Args:
            solution (dict): solution
        
        Returns:
            dict: best schedule
        """
        self.update_feasible_schedules(timetable)
        best_schedule = None
        best_revenue = -np.inf
        for fs in self.feasible_schedules:
            revenue = self.get_revenue({'real': timetable['real'], 'discrete': fs}, update_schedule=False)
            if revenue > best_revenue:
                best_revenue = revenue
                best_schedule = fs
        return np.array(best_schedule)
    
    def get_fitness_gsa(self, timetable):
        """
        Get fitness
        
        Args:
            solution (dict): solution
        
        Returns:
            float: fitness
        """
        best_schedule = self.get_best_schedule(timetable)
        return self.get_revenue({'real': timetable['real'], 'discrete': best_schedule}, update_schedule=False)
    
    def feasible_services_times(self, timetable):
        """
        Check if the service is feasible
        
        Returns:
            bool: True if the service is feasible, False otherwise
        """
        self.update_schedule(timetable)
        if not self._feasible_boundaries(timetable):
            return False
        
        scheduling = np.ones(len(self.requested_schedule))
        if  self._travel_times_feasibility(scheduling) and self._stop_times_feasibility(scheduling):
            return True
        return False
            


In [49]:
schedule = get_schedule_request(20)

In [50]:
# Dummy schedule
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

In [51]:
from scipy.stats import loguniform

np.random.seed(seed=22)

def get_revenue_behaviour(schedule: dict) -> dict:
    """
    Get revenue behaviour
    
    Args:
        schedule (dict): schedule
    
    Returns:
        dict: revenue behaviour
    """
    revenue = {}
    bias = [0.2, 0.35, 0.1]
    for service in schedule:
        b = np.random.choice(bias)
        base_price = 55 * len(schedule[service])
        canon = base_price + b * base_price
        k = loguniform.rvs(0.01, 100, 1)
        max_penalty = canon * 0.4
        dt_penalty = max_penalty * 0.35
        tt_penalty = (max_penalty - dt_penalty) / (len(schedule[service]) - 1)
        revenue[service] = {'canon': canon, 'k': k, 'dt_max_penalty': dt_penalty, 'tt_max_penalty': tt_penalty}
    return revenue

revenue = get_revenue_behaviour(schedule)
revenue

{1: {'canon': 222.75,
  'k': 7.424686268142557,
  'dt_max_penalty': 31.185000000000002,
  'tt_max_penalty': 28.957500000000003},
 2: {'canon': 264.0,
  'k': 1.4810078247291318,
  'dt_max_penalty': 36.96,
  'tt_max_penalty': 22.880000000000006},
 3: {'canon': 181.5,
  'k': 14.186717826602765,
  'dt_max_penalty': 25.41,
  'tt_max_penalty': 23.595000000000006}}

In [52]:
sm = RevenueMaximization(schedule, revenue, safe_headway=10)

In [53]:
gsa_algo = GSA(objective_function=sm.get_fitness_gsa,
               is_feasible=sm.feasible_services_times,
               r_dim=len(sm.real_boundaries),
               d_dim=0,
               boundaries=sm.boundaries)

In [54]:
gsa_algo.set_seed(seed=28)

training_history = gsa_algo.optimize(population_size=5,
                                     iters=300,
                                     chaotic_constant=False,
                                     repair_solution=True)

Initializing positions of the individuals in the population...
Initialized individuals:  1
Initialized individuals:  2
Initialized individuals:  3
Initialized individuals:  4
Initialized individuals:  5
{'real': [array([ -7.94597813, 154.25540342, 159.34878163, 189.72870956,
         5.10050016,  25.81246663,  28.44843274, 166.03381484,
       172.12106335, 218.7311867 ,  27.56868617, 180.34337858,
       190.82174754, 233.91010781]), array([ -8.80091106, 140.6990981 , 146.15742589, 186.00796707,
         1.31919856,  25.91579399,  33.98481627, 170.32691459,
       172.55065803, 218.78132393,  22.97751784, 178.09895922,
       182.22930455, 230.03199336]), array([ -3.15375178, 147.55131748, 157.31592311, 187.76222326,
        -1.27994096,  27.01872099,  34.91712479, 171.39623989,
       174.41800132, 218.62501662,  23.74251788, 176.73248939,
       184.3441696 , 230.57135233]), array([-7.62645716e+00,  1.42114819e+02,  1.54038747e+02,  1.82313778e+02,
        4.29745524e-02,  2.3999827

/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.

Repairing solution...
Repairing solution...
['At iteration 3 the best fitness is 590.9294483996739']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 4 the best fitness is 590.9294483996739']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 5 the best fitness is 590.9294483996739']
Repairing solution...
Repairing solution...


/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.

Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 6 the best fitness is 590.9294483996739']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 7 the best fitness is 590.9294483996739']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 8 the best fitness is 590.9294483996739']
Repairing solution...


...

Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 91 the best fitness is 619.1668318243312']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 92 the best fitness is 619.2158261570023']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 93 the best fitness is 619.5210607456729']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 94 the best fitness is 619.7487594555444']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...


/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.

['At iteration 95 the best fitness is 619.9101609643244']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 96 the best fitness is 620.0596682699041']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 97 the best fitness is 620.4182754387302']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 98 the best fitness is 620.6223925689319']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 99 the best fitness is 620.7654193596335']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...


/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/david/PycharmProjects/GSA_M/venv/lib/python3.

['At iteration 100 the best fitness is 621.1839334662466']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 101 the best fitness is 621.4691623827298']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 102 the best fitness is 622.1778550419733']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 103 the best fitness is 622.2677026152577']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 104 the best fitness is 622.4681668348259']
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 105 the best fitness is 622.5239522772206']
Repairing solution...
Repairing solution...
['At iteration 106 the best fitness is 622.6577580954447']
Repairing solution...
Repairing solution...
['At iteration 

In [55]:
sum([revenue[service]['canon'] for service in revenue])

668.25

In [117]:
training_history

,Iteration,Fitness,ExecutionTime,Discrete,Real
0,0,192.027224,0.001174,"[1, 0, 0]","[-6.3461328316835, 144.65157610387462, 155.975..."
1,1,350.470332,0.101916,"[1, 0, 1]","[-4.8656529980557925, 144.0706120785215, 153.0..."
2,2,350.470332,0.419042,"[1, 0, 1]","[-4.8656529980557925, 144.0706120785215, 153.0..."
3,3,350.470332,0.650546,"[1, 0, 1]","[-4.8656529980557925, 144.0706120785215, 153.0..."
4,4,350.470332,0.844651,"[1, 0, 1]","[-4.8656529980557925, 144.0706120785215, 153.0..."
...,...,...,...,...,...
295,295,379.365826,13.331366,"[1, 0, 1]","[0.011904464460581939, 150.10248532905757, 156..."
296,296,379.365826,13.335702,"[1, 0, 1]","[0.011904464071080438, 150.10248532920832, 156..."
297,297,379.365826,13.338657,"[1, 0, 1]","[0.011904463958705438, 150.10248532923146, 156..."
298,298,379.365826,13.341262,"[1, 0, 1]","[0.011904463817553576, 150.10248532923927, 156..."


In [118]:
for row in training_history.iterrows():
    sol_dis = list(row[1]['Discrete'])
    sol_real = list(row[1]['Real'])
    print(sm.get_revenue({'real': np.array(sol_real), 'discrete': np.array(sol_dis)}))
    

192.0272237680468
350.4703320438339
350.4703320438339
350.4703320438339
350.4703320438339
350.4703320438339
350.4703320438339
355.1444651642497
378.43994679619436
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.8259082813064
378.82590

In [61]:
revenue

{1: {'canon': 222.75,
  'k': 7.424686268142557,
  'dt_max_penalty': 31.185000000000002,
  'tt_max_penalty': 28.957500000000003},
 2: {'canon': 264.0,
  'k': 1.4810078247291318,
  'dt_max_penalty': 36.96,
  'tt_max_penalty': 22.880000000000006},
 3: {'canon': 181.5,
  'k': 14.186717826602765,
  'dt_max_penalty': 25.41,
  'tt_max_penalty': 23.595000000000006}}

In [ ]:
# S_i ES UNA CONSECUENCIA DE LOS TIEMPO, NO UNA VARIABLE EN SÍ MISMA.